In [ ]:
import torch
from torch import nn, save, load
from torch.optim import Adam
from torch.utils.data import sampler, DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from google.colab import drive
from PIL import Image
drive.mount('/content/drive')

train = datasets.CIFAR10(root="data", download=True , train=True, transform=ToTensor())
train_loader = DataLoader(train, batch_size=30, sampler=sampler.RandomSampler(train))

class ImageClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.model = nn.Sequential(
           nn.Conv2d(3, 16, (3,3), padding=2),
           nn.ReLU(),
           nn.MaxPool2d(2,2),
           nn.Dropout(0.25),
           nn.Conv2d(16, 32, (3,3), padding=2),
           nn.ReLU(),
           nn.Conv2d(32, 64, (3,3), padding=2),
           nn.ReLU(),
           nn.MaxPool2d(2,2),
           nn.Dropout(0.25),
           nn.Conv2d(64, 128, (3,3), padding=2),
           nn.ReLU(),
           nn.Flatten(),
           nn.Linear(128 * 12 * 12, 10)
        )

    def forward(self, x):
        return self.model(x)

def get_class_label(prediction):
    class_labels = ["Pesawat", "Mobil", "Burung", "Kucing", "Rusa", "Anjing", "Katak", "Kuda", "Kapal", "Truk"]
    return class_labels[prediction]

clf = ImageClassifier().to('cuda')
opt = Adam(clf.parameters(), lr=1e-4, weight_decay=0.001)
loss_fn = nn.CrossEntropyLoss()

if __name__ == "__main__":
    total_correct = 0
    total_samples = 0

    for epoch in range(200):
        clf.train()
        for batch in train_loader:
            X, y = batch
            X, y = X.to('cuda'), y.to('cuda')
            yhat = clf(X)
            loss = loss_fn(yhat, y)

            opt.zero_grad()
            loss.backward()
            opt.step()


        clf.eval()
        with torch.no_grad():
            for batch in train_loader:
                X, y = batch
                X, y = X.to('cuda'), y.to('cuda')
                yhat = clf(X)
                _, predictions = yhat.max(1)
                total_correct += (predictions == y).sum().item()
                total_samples += predictions.size(0)

        accuracy = (total_correct / total_samples) * 100
        print(f"Epoch:{epoch} loss is {loss.item()}, accuracy is {accuracy:.2f}%")

    with open('model_state_pt', 'wb') as f:
        save(clf.state_dict(), f)

    # test
    # with open('model_state_pt', 'rb') as f:
    #     clf.load_state_dict(load(f))

    # img = Image.open('anjing.jpeg')
    # img_tesor = ToTensor()(img).unsqueeze(0).to('cuda')

    # prediction = torch.argmax(clf(img_tesor)).item()``
    # class_label = get_class_label(prediction)

    # print(f"{class_label}")

100%|██████████| 170498071/170498071 [00:04<00:00, 39235424.71it/s]


Extracting data/cifar-10-python.tar.gz to data
Epoch:0 loss is 1.5518906116485596, accuracy is 42.95%
Epoch:1 loss is 0.9876619577407837, accuracy is 45.29%
Epoch:2 loss is 1.6202256679534912, accuracy is 47.54%
Epoch:3 loss is 1.371886134147644, accuracy is 49.24%
Epoch:4 loss is 1.0148786306381226, accuracy is 50.79%
Epoch:5 loss is 1.1551307439804077, accuracy is 51.88%
Epoch:6 loss is 1.289198637008667, accuracy is 53.07%
Epoch:7 loss is 1.0139539241790771, accuracy is 53.88%
Epoch:8 loss is 1.1257312297821045, accuracy is 54.90%
Epoch:9 loss is 1.1705992221832275, accuracy is 55.82%
Epoch:10 loss is 1.0766315460205078, accuracy is 56.66%
Epoch:11 loss is 1.1747199296951294, accuracy is 57.44%
Epoch:12 loss is 1.5306518077850342, accuracy is 58.13%
Epoch:13 loss is 0.8006302714347839, accuracy is 58.92%
Epoch:14 loss is 1.1394822597503662, accuracy is 59.63%
Epoch:15 loss is 1.6408506631851196, accuracy is 60.29%
Epoch:16 loss is 0.9517695307731628, accuracy is 60.93%
Epoch:17 loss

In [ ]:
pip list

Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2.0
beautifulsoup4                   4.12.3
bi